            GENERERATIVE CHATBOT MADE w/ KERAS, RNN (LSTM) AND TRAINED w/ SARC DS.

Keep in mind this documents the code, some parts might not be in the correct order, for the working code please go to the XXX document.

### REQUIREMENTS

#### LIBRARIES

In [1]:
#Basic libraries to import:
import numpy as np  #used for scientific computing
import pandas as pd #for data manipulation and analysis - used to upload de DS we are working with.

#NLP
import nltk # Natural Language Toolkit, platform for building Python programs to work with human language data.

#nltk.download('punkt') # tokenizer that divides a text into a list of sentences

from collections import Counter 

from keras.models import Model
from keras.models import load_model 

from keras.layers import Dense, Input, Embedding

from keras.layers.recurrent import LSTM

from keras.preprocessing.sequence import pad_sequences

from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback

from sklearn.model_selection import train_test_split



Using TensorFlow backend.


#### PARAMETERS

In [2]:
# set default parameters
BATCH_SIZE = 128 # number of samples processed before the model is updated. 
NUM_EPOCHS = 500 # number of complete passes through the training dataset. For generative models we need a lot of epochs so the model can learn, we will see if 500 is enough
HIDDEN_UNITS = 100 #number of hidden layers, they perform nonlinear transformations of the inputs entered into the network.
MAX_INPUT_SEQ_LENGTH = 20 # max. number of words the chatbot will consider as input
MAX_TARGET_SEQ_LENGTH = 20 # max. number of words the chatbot will reply with
MAX_VOCAB_SIZE = 20000 #10-20k  https://coursefinders.com/blog/es/5669/espanol-cuantas-palabras-se-necesitan-para-hablar-con-fluidez-un-idioma

### INPUT

In [ ]:
# read the data
import pandas as pd 

df = pd.read_csv('SARC_DS.csv')
lines = df['all']

# Containers that keeps track of how many times equivalent values are added.
# Used for the text preprocessing and the encoding process
from collections import Counter 

input_counter = Counter()
target_counter = Counter()

#create the vocabulary from the dataset to train the model
input_texts = []
target_texts = []
prev_words = []

import nltk 

for line in lines:

    next_words = [w.lower() for w in nltk.word_tokenize(line) if w.isalpha()]

    if len(next_words) > MAX_TARGET_SEQ_LENGTH:
        next_words = next_words[0:MAX_TARGET_SEQ_LENGTH]

    if len(prev_words) > 0:
        input_texts.append(prev_words)
        
        for w in prev_words:
            input_counter[w] += 1
            
        target_words = next_words[:]
        target_words.insert(0, 'START')
        target_words.append('END')
        
        for w in target_words:
            target_counter[w] += 1
            
        target_texts.append(target_words)

    prev_words = next_words

In [ ]:
# encode the data

input_word2idx = dict()
target_word2idx = dict()

for idx, word in enumerate(input_counter.most_common(MAX_VOCAB_SIZE)):
    input_word2idx[word[0]] = idx + 2
    
for idx, word in enumerate(target_counter.most_common(MAX_VOCAB_SIZE)):
    target_word2idx[word[0]] = idx + 1

In [3]:
import pickle
with open('input_word2idx.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    input_word2idx = pickle.load(f)

with open('target_word2idx.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    target_word2idx = pickle.load(f)

input_word2idx['PAD'] = 0
input_word2idx['UNK'] = 1
target_word2idx['UNK'] = 0
input_idx2word = dict([(idx, word) for word, idx in input_word2idx.items()])
target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

num_encoder_tokens = len(input_idx2word)
num_decoder_tokens = len(target_idx2word)


In [ ]:
encoder_input_data = []

encoder_max_seq_length = 0
decoder_max_seq_length = 0

for input_words, target_words in zip(input_texts, target_texts):
    encoder_input_wids = []
    
    for w in input_words:
        w2idx = 1
        
        if w in input_word2idx:
            w2idx = input_word2idx[w]
            
        encoder_input_wids.append(w2idx)

    encoder_input_data.append(encoder_input_wids)
    encoder_max_seq_length = max(len(encoder_input_wids), encoder_max_seq_length)
    decoder_max_seq_length = max(len(target_words), decoder_max_seq_length)

    
context = dict()
context['num_encoder_tokens'] = num_encoder_tokens
context['num_decoder_tokens'] = num_decoder_tokens
context['encoder_max_seq_length'] = encoder_max_seq_length
context['decoder_max_seq_length'] = decoder_max_seq_length

In [ ]:
def generate_batch(input_data, output_text_data, BATCH_SIZE):
    '''
    Custom function to generate batches
    
    input: 
        - input_data 
        - output_text_data
        - BATCH_SIZE
        
    output:
        - generator object
    '''
    
    num_batches = len(input_data) // BATCH_SIZE
    
    while True:
        for batchIdx in range(0, num_batches):
            start = batchIdx * BATCH_SIZE
            end = (batchIdx + 1) * BATCH_SIZE
            
            encoder_input_data_batch = pad_sequences(input_data[start:end], encoder_max_seq_length)
            
            decoder_target_data_batch = np.zeros(shape=(BATCH_SIZE, decoder_max_seq_length, num_decoder_tokens))
            decoder_input_data_batch = np.zeros(shape=(BATCH_SIZE, decoder_max_seq_length, num_decoder_tokens))
            
            for lineIdx, target_words in enumerate(output_text_data[start:end]):
                for idx, w in enumerate(target_words):
                    w2idx = 0
                    
                    if w in target_word2idx:
                        w2idx = target_word2idx[w]
                    decoder_input_data_batch[lineIdx, idx, w2idx] = 1
                    
                    if idx > 0:
                        decoder_target_data_batch[lineIdx, idx - 1, w2idx] = 1
            
            yield [encoder_input_data_batch, decoder_input_data_batch], decoder_target_data_batch

### MODEL CREATION

#### LAYERS

##### keras.Input()
It's used to instantiate a Keras tensor; a TensorFlow symbolic tensor object, which we augment with certain attributes that allow us to build a Keras model just by knowing the inputs and outputs of the model. https://keras.io/api/layers/core_layers/input/
        
    Used Arguments:
- **shape**: shape tuple (integers), not including the batch size. Elements of this tuple can be None; 'None' elements represent dimensions where the shape is not known.<br>
- **name**: String, the name of the layer. (opt)
<br>


    Returns:
- Tensor.

In [ ]:
from keras.layers import Input

encoder_inputs = Input(shape=(None,), 
                       name='encoder_inputs')

decoder_inputs = Input(shape=(None, num_decoder_tokens), 
                       name='decoder_inputs')

decoder_state_inputs = [Input(shape=(HIDDEN_UNITS,)), 
                        Input(shape=(HIDDEN_UNITS,))]

##### keras.Embedding()
Turns positive integers (indexes) into dense vectors of fixed size. This layer can only be used as the first layer in a model. https://keras.io/api/layers/core_layers/embedding/
        
    Used Arguments:
- **input_dim**: Integer. Size of the vocabulary, i.e. maximum integer index + 1.<br>
- **output_dim**: Integer. Dimension of the dense embedding.<br>
- **input_length**: Length of input sequences, when it is constant. This argument is required if you are going to connect Flatten then Dense layers upstream (without it, the shape of the dense outputs cannot be computed).<br>
- **name**: String, the name of the layer. (opt)
<br>


    Input shape:
- 2D tensor with shape: (batch_size, input_length).
<br>


    Output shape:    
- 3D tensor with shape: (batch_size, input_length, output_dim).

In [ ]:
from keras.layers import Embedding

encoder_embedding = Embedding(input_dim=num_encoder_tokens, 
                              output_dim=HIDDEN_UNITS,
                              input_length=encoder_max_seq_length, 
                              name='encoder_embedding')

##### keras.LSTM()
*(Long Short-Term Memory layer - Hochreiter 1997.)*
Based on available runtime hardware and constraints, this layer will choose different implementations (cuDNN-based or pure-TensorFlow) to maximize the performance. https://keras.io/api/layers/recurrent_layers/lstm/

        
    Used Arguments:
    
- **units**: Positive integer, dimensionality of the output space.<br>
- **return_sequences**: Boolean. Whether to return the last output. in the output sequence, or the full sequence. Default: 'False'.<br>
- **return_state**: Boolean. Whether to return the last state in addition to the output. Default: 'False'.<br>
- **name**: String, the name of the layer. (opt) <br>
<br>


    Used Called Arguments:
    
- **initial_state**: List of initial state tensors to be passed to the first call of the cell (optional, defaults to None which causes creation of zero-filled initial state tensors).<br>

In [ ]:
from keras.layers.recurrent import LSTM

encoder_lstm = LSTM(units=HIDDEN_UNITS, 
                    return_state=True, 
                    name='encoder_lstm')

encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(
                                                encoder_embedding(
                                                    encoder_inputs))

encoder_states = [encoder_state_h, encoder_state_c]

decoder_lstm = LSTM(units=HIDDEN_UNITS, 
                    return_state=True, 
                    return_sequences=True, 
                    name='decoder_lstm')

decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(
                                        decoder_inputs,
                                        initial_state=encoder_states)

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, 
                                                 initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]

##### keras.layers.Dense()
Dense layer is the regular deeply connected neural network layer. https://keras.io/api/layers/core_layers/dense/
        
    Arguments
- **units**: Number of units and it affects the output layer.<br>
- **activation**: Activation function. <br>
- **name**: String, the name of the layer. (opt)
<br>


    Input shape:
- N-D tensor with shape: (batch_size, ..., input_dim). The most common situation would be a 2D input with shape (batch_size, input_dim).
<br>


    Output shape:    
- N-D tensor with shape: (batch_size, ..., units). For instance, for a 2D input with shape (batch_size, input_dim), the output would have shape (batch_size, units).

In [ ]:
from keras.layers import Dense

decoder_dense = Dense(
                units=num_decoder_tokens,
                activation='softmax', #converts a real vector to a vector of categorical probabilities
                name='decoder_dense'
                )
decoder_outputs = decoder_dense(decoder_outputs)

#### OBJECT CREATION & MODEL CONFIGURATION

##### keras.Model()
Model groups layers into an object with training and inference features. https://keras.io/api/models/model/#model-class

    Arguments
- **inputs**: The input(s) of the model: a `keras.Input` object or list of `keras.Input` objects. <br>
- **outputs**: The output(s) of the model. <br>
- **name**: String, the name of the model. (opt) <br>

In [ ]:
from keras.models import Model,

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

encoder_model = Model(encoder_inputs, encoder_states)

decoder_model = Model([decoder_inputs] + decoder_state_inputs, 
                      [decoder_outputs] + decoder_states)

##### keras.model.compile()
To configurate the model with loses and optimizers, by default. You can also add metrics
    
    Losses: 

The purpose is to compute the quantity that a model should seek to minimize during training.
https://keras.io/api/losses/<br>

- **CategoricalCrossentropy**: Default. Computes the cross entropy loss between the labels and predictions; when there are two or more label classes.
<br>
       
       
    Optimizers: 

An optimizer is one of the two arguments required for compiling a Keras model, it's objective... optimize: https://keras.io/api/optimizers/<br>

- **Adam**: Default. Stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments. /it works for generative models because according to Kingma et al., 2014, 'is well suited for problems that are large in terms of data'/
<br>
    
    
    Metrics: /not used/

Used to judge the performance of your model.
https://keras.io/api/metrics/

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()
#Compiling
#training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#training_model.summary()

#### MODEL TRAINING PREP

##### sklearn.model_selection.train_test_split()

Split arrays or matrices into random train and test subsets. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
    
    Used Parameters: 

- ***arrays**: sequence of indexables with same length.<br>
- ***test_size**: proportion of the dataset to include in the test split.<br>
- ***train_size**: proportion of the dataset to include in the train split.<br>
- ***random_state**: parameter to control the random number generator used. /For more information about random_state check out: https://scikit-learn.org/stable/glossary.html#term-random-state /
<br>
       
       
    Returns: 

- **splitting**: list, length=2 * len(arrays): X_train, X_test, y_train, y_test /Careful with the order/

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(encoder_input_data, 
                                                    target_texts, 
                                                    test_size=0.2, 
                                                    random_state=42)

train_gen = generate_batch(X_train, y_train, BATCH_SIZE)
test_gen = generate_batch(X_test, y_test, BATCH_SIZE)

train_num_batches = len(X_train) // BATCH_SIZE
test_num_batches = len(X_test) // BATCH_SIZE

##### keras.callbacks.ModelCheckpoint()

Callback to save the Keras model or model weights at some frequency. https://keras.io/api/callbacks/model_checkpoint/
    
    Used Arguments: 

- **filepath**: string or PathLike, path to save the model file.<br>
- **monitor**: quantity to monitor.<br>
- **verbose**:verbosity mode, 0 or 1.<br>
- **save_best_only**:when save_best_only=True, the latest best model according to the quantity monitored will not be overwritten. <br>
- **mode**: For 'loss' this should be min.<br>
- ****kwargs**: Additional arguments for backwards compatibility. Possible key is period.
<br>


In [11]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback

checkpoint = ModelCheckpoint(
                            'model_best_weights', #name of the document where the checkpoints will be saved
                            monitor='loss', 
                            verbose=1, 
                            save_best_only=True, 
                            mode='min', 
                            period = 3 #saved every 3 epochs when loss improves
                            )

my_callbacks = [checkpoint]

### 

######
To train my model with the established vocabulary of 20k words and 500 epochs; and starting with 40k lines of my ds it would take an estimate of 17 days to train in my local machine.

Epoch 1/500
  8/312 [..............................] - ETA: 53:45 - loss: 9.0315
  
 Using a VM the same model with the same parameters takes an estimate of 5 days.
 
 Epoch 1/500
249/249 [==============================] - 923s 4s/step - loss: 6.7226 - val_loss: 6.4012

To avoid unnecessary training I am using the callback attribute with `Earlystop` which will automatically stop the training once it has stopped improving, this means that it might take less time. We will see, today is 31.May.2020 01:19-

-- **`40k lines of training is insufficient to have a properly trained model`**

At the same time, given that it takes days to train, to avoid any technical issues I am using the `ModelCheckpoint` function which saves a copy of the model, in this case it is monitoring the 'loss', which on second though might not be the correct one and it should be monitoring the 'val_loss', but the main thing is that it makes a "Security Copy" of the model so if the Server of the VM were to disconnect or the Kernel were to die I'd have a least part of the model saved.

Because we are working with NLG the training has to be very extense. That's why I'm creating different models, which might not be completed for due date but will be useful to get a general idea and even continue training that model.

#### MODEL TRAINING

##### fit_generator()

**Model.fit now supports generators, so there is no longer any need to use this endpoint.**

Fits the model on data yielded batch-by-batch by a Python generator. https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit // https://kite.com/python/docs/tensorflow.keras.Model.fit_generator
    
    Used Arguments: 

- **generator**: A generator object, created from a tuple `(inputs, targets)`, in order to avoid duplicate data when using multiprocessing.<br>
- **steps_per_epoch**: Total number of steps (batches of samples) to yield from `generator` before declaring one epoch finished and starting the next epoch.<br>
- **epochs**: Integer, total number of iterations on the data.<br>
- **verbose**:Verbosity mode, 0, 1, or 2; used for each epochs progress visualization <br>
- **callbacks**: List of callbacks to be called during training.<br>
- **validation_data**: generator for the validation data.<br>
- **validation_steps**:Total number of steps (batches of samples) to yield from `generator` before stopping.
<br>
    
    
    Retruns:


- A `History` object.

In [ ]:
model_tot_500e = model.fit_generator(generator=train_gen,
                    steps_per_epoch=train_num_batches,
                    epochs= NUM_EPOCHS,
                    verbose=1,
                    validation_data=test_gen,
                    validation_steps=test_num_batches,
                    callbacks = my_callbacks
                   )

### SAVING & LOADING

In [ ]:
model.save("file_name")

In [ ]:
from keras.models import load_model

model = load_model('file_name')

### OUTPUT

In [ ]:
input_text = input()
input_seq = []
input_wids = []
max_encoder_seq_length = 20
max_decoder_seq_length = 20

for word in nltk.word_tokenize(input_text.lower()):
    idx = 1
    if word in input_word2idx:
        idx = input_word2idx[word]
    input_wids.append(idx)
    
input_seq.append(input_wids)

##### pad_sequence()

Pads sequences to the same length, in other words, the function transforms a list of sequences (lists of integers) into a 2D Numpy array of shape (num_samples, maxlen). 
https://keras.io/api/preprocessing/timeseries/#pad_sequences-function
    
    Used Arguments: 

- **sequences**: List of sequences (each sequence is a list of integers).<br>
- **maxlen**: Optional Int, maximum length of all sequences. If not provided, sequences will be padded to the length of the longest individual sequence.
<br>
    
    
    Returns: 

- Numpy array with shape (len(sequences), maxlen)


In [ ]:
from keras.preprocessing.sequence import pad_sequences

input_seq = pad_sequences(input_seq, 
                          maxlen= max_encoder_seq_length)

In [ ]:
states_value = encoder_model.predict(input_seq)
target_seq = np.zeros((1, 1, num_decoder_tokens))
target_seq[0, 0, target_word2idx['START']] = 1
target_text = ''
target_text_len = 0
terminated = False

while not terminated:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    sample_token_idx = np.argmax(output_tokens[0, -1, :])
    sample_word = target_idx2word[sample_token_idx]
    target_text_len += 1

    if sample_word != 'START' and sample_word != 'END':
        target_text += ' ' + sample_word

    if sample_word == 'END' or target_text_len >= max_decoder_seq_length:
        terminated = True

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sample_token_idx] = 1

    states_value = [h, c]
    

target_text.strip().replace('UNK', '')